<a href="https://colab.research.google.com/github/thess005/Programming-Collective-Intelligence/blob/master/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret pandas shap lxml

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import pandas as pd
import glob
import os
import re

if not os.path.exists('/content/save'):
    os.makedirs('/content/save')

folder_path = '/content/W600'
file_list = glob.glob(folder_path + "/*.html")
main_dataframe = pd.DataFrame(pd.read_html(file_list[0]))

for i in range(0,len(file_list)):
    data = pd.read_html(file_list[i])
    df = pd.DataFrame(data[22],columns=['Description', 'Occurrences'])
    main_dataframe = pd.concat([df],axis=0)
    filename = file_list[i]
    match_object = re.match(r'^.*/(.*)/(.*)/(.*)@(.*)@(.*)@(.*)\.html$', filename)
    cwd = os.getcwd()
    path = cwd + '/save/'
    savefile = (match_object.group(4)+'.csv')
    print(df)
    df.to_csv(path+savefile, encoding='utf-8', index=False)

In [40]:
import pandas as pd
import glob
import os
import re

folder_path = '/content/W600'
file_list = glob.glob(folder_path + "/*.html")
main_dataframe = pd.DataFrame(pd.read_html(file_list[0]))
df = pd.DataFrame()
for i in range(0,len(file_list)):
    data = pd.read_html(file_list[i])
    time = pd.DataFrame(data[1],columns=['Planned Shift Start'])
    stops = pd.DataFrame(data[22],columns=['Description', 'Occurrences'])
    main_dataframe = pd.concat([time, stops])
    main_dataframe = main_dataframe.fillna(method='ffill')
    df=df.append(main_dataframe,ignore_index=True)

#main_dataframe = pd.concat([time, stops])
#print(df)
df.to_csv('data.csv', encoding='utf-8', index=False)






In [ ]:
!pip install pycaret==2.3.5
!pip install scipy==1.4.1
!pip install pyyaml==5.4.1

In [160]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from jupyterthemes import jtplot
jtplot.style()

from pycaret.utils import enable_colab
enable_colab()

from pycaret.datasets import get_data


data = pd.read_csv('data.csv')

data.shape

data.head()




print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

from pycaret.anomaly import *
print (data)
exp_ano101 = setup(data, normalize = True,session_id = 123)

iforest = create_model('iforest')

print(iforest)

svm = create_model('svm', fraction = 0.025)

print(svm)

iforest_results = assign_model(iforest)
print(iforest_results[['Anomaly']])
print(iforest_results.loc[iforest_results['Anomaly'] == 1])



OCSVM(cache_size=200, coef0=0.0, contamination=0.025, degree=3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False)
     Anomaly
0          0
1          0
2          0
3          0
4          0
..       ...
365        0
366        0
367        0
368        0
369        0

[370 rows x 1 columns]
       Planned Shift Start                            Description  \
6      7/6/2021 4:00:00 PM                     STOP FROM CARTONER   
56    6/28/2021 4:00:00 PM                PACKET CONVEYOR IN STOP   
94   6/30/2021 12:00:00 AM                     STOP FROM CARTONER   
95   6/30/2021 12:00:00 AM             FAIL: EXITING PACKET CHECK   
96   6/30/2021 12:00:00 AM    FILM CENTERING MARK OUT OF POSITION   
107   6/30/2021 4:00:00 PM                     STOP FROM CARTONER   
120    7/1/2021 8:00:00 AM                     STOP FROM CARTONER   
121    7/1/2021 8:00:00 AM                INEFF. PACKET REJECTION   
122    7/1/2021 8:00:00 AM            

In [148]:
plot_model(iforest)
print(iforest.predict_confidence)

<bound method BaseDetector.predict_confidence of IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)>


In [149]:
anomaly = iforest_results.loc[iforest_results['Anomaly'] == 1]
time = anomaly[['Planned Shift Start']]
print(time)

       Planned Shift Start
6      7/6/2021 4:00:00 PM
56    6/28/2021 4:00:00 PM
94   6/30/2021 12:00:00 AM
95   6/30/2021 12:00:00 AM
96   6/30/2021 12:00:00 AM
107   6/30/2021 4:00:00 PM
120    7/1/2021 8:00:00 AM
121    7/1/2021 8:00:00 AM
122    7/1/2021 8:00:00 AM
153   6/30/2021 8:00:00 AM
158   6/30/2021 8:00:00 AM
160   6/30/2021 8:00:00 AM
174   6/10/2021 4:00:00 PM
203    7/1/2021 4:00:00 PM
204    7/1/2021 4:00:00 PM
209    7/1/2021 4:00:00 PM
242   7/1/2021 12:00:00 AM
245   7/1/2021 12:00:00 AM
261    7/6/2021 8:00:00 AM


In [150]:
stopDate = time['Planned Shift Start'].str.split(' ').str[0]
print(stopDate)

6       7/6/2021
56     6/28/2021
94     6/30/2021
95     6/30/2021
96     6/30/2021
107    6/30/2021
120     7/1/2021
121     7/1/2021
122     7/1/2021
153    6/30/2021
158    6/30/2021
160    6/30/2021
174    6/10/2021
203     7/1/2021
204     7/1/2021
209     7/1/2021
242     7/1/2021
245     7/1/2021
261     7/6/2021
Name: Planned Shift Start, dtype: object


In [151]:
changeDate = pd.read_xml('AF.xml')
changeDate = (changeDate['dateTime'].str.split(' ').str[0])

In [152]:
for i in stopDate:
  for j in changeDate:
    if i == j:
        print(i)

In [ ]:
iforest_results.loc[iforest_results['Anomaly'] == 1]